# Laboratorio 1 - Informe

### Grupo 4:
     - S. Calvo C.I 5.711.417-7     
     - X. Iribarnegaray C.I
     - J. Simonelli C.I 5.405.358-4

## 1. Objetivo

El objetivo de este laboratorio es:
- Implementar el algoritmo ID3, añadiendo el hiperparámetro *max_range_split*, que determina la cantidad máxima de rangos en los que se puede partir un atributo númerico.
- Utilizar scikit-learn para el preprocesamiento de datos y la creación de modelos basados en árboles de decisión.
- Evaluar y comparar los modelos generados.

## 2. Diseño

### 2.1 Preprocesamiento de datos
Previo a la ejecución del algoritmo se es provisto a este, mediante una variable global, el conjunto de feature values continuos. Luego, durante la ejecución del algoritmo, se discretizaran los valores asociados a estos feature values.


### 2.2 Algoritmo
En primer lugar, definimos la función `entropy(dataset, target)` que será utilizada a lo largo de la implementación del algoritmo ID3:


In [5]:
import numpy as np

def entropy(dataset, target):
    # value_counts() returns a Series containing the counts of unique values
    values = dataset[target].value_counts()
    # shape returns the size of dataset, shape[0] being the number of rows
    total = dataset.shape[0]
    p0 = values.iloc[0]/total
    if (len(values) > 1):
        p1 = values.iloc[1]/total
        return -(p0)*np.log2(p0) - (p1) * np.log2(p1)
    else: 
        return -(p0)*np.log2(p0)

Luego, el siguiente paso a delinear en el algoritmo es la obtención del "mejor atributo". Esto lo logramos mediante la función\
`best_feature(dataset, target, features, continuous_features, max_range_splits)`\
que retorna el siguiente mejor atributo a elegir. Esta decisión es hecha en base a la fórmula de Ganancia vista en el curso, donde, para maximizar la ganancia, basta con minimizar el segundo factor de la fórmula, y, por lo tanto, se eligirá el atributo que minimice este valor.


In [6]:
from id3 import actually_split
def best_feature(dataset, target, features, continuous_features, max_range_splits):
    entropies = []
    continuous = {}
    for feature in features:
        if feature in continuous_features:
            # Continuous-Valued feature 
            aux_entropy, best_split = get_splits(dataset, feature, target, max_range_splits)
            entropies.append(aux_entropy)
            continuous[feature] = best_split
        else :
            # Discrete-Valued feature
            res = 0
            for value, count in dataset[feature].value_counts().items():
                res += count*entropy(dataset.loc[dataset[feature] == value], target)
            entropies.append(res / dataset.shape[0])
    best_feature = features[entropies.index(min(entropies))]
    
    if not (best_feature in continuous):
        return best_feature, dataset
    return best_feature, actually_split(dataset.copy(), best_feature, continuous[best_feature])

Para el caso en el que el atributo a evaluar sea discreto (`else` del `if`), solamente se lleva a cabo el cálculo descrito previamente.\
Por otro lado, para el caso de evaluar un atributo de valores continuos, se debe realizar otro procedimiento.


Dado que por defecto el algoritmo ID3 solamente aplica a ejemplares de valores discretos, en el caso de querer entrenar un modelo mediante un dataset que contiene valores continuos, será necesario discretizarlos en tiempo de ejecución.\
Esto es logrado mediante la división de los valores continuos, con valores calculados con el promedio de dos puntos con valores objetivo distintos. Luego, dependiendo del valor del hiperparámetro *max_range_splits* (de valor 2 o 3 ), utilizamos combinaciones de estos puntos para particionar los valores continuos en rangos. De esta manera, obtenemos finalmente un atributo de valores discretos.\
\
Sin embargo se debe tener en cuenta que, en datasets de gran tamaño, es probable que exista una gran cantidad de posibles puntos de corte, por lo cual es necesario identificar el mejor o mejor par de puntos. De manera similar al cálculo del "mejor atributo", se realiza un cálculo de la ganancia para cada split, y se elige el que minimice el segundo factor de la fórmula. Para esto, se utiliza la función 
`get_splits(dataset, feature, target, max_range_splits)`

In [7]:
from id3 import generate_every_pair_from_list
from id3 import discretize_dataset

def get_splits(dataset, feature, target, max_range_splits):
    min_entropy = 2
    dataset = dataset.sort_values(by=feature)
    current_target = dataset[target].iloc[0]
    dataset_size = dataset.shape[0]
    candidate_splits = []
    best_values = []
    
    # Finding splits, iterating through the dataset rows
    for i in range(1, dataset_size):
        if current_target != dataset[target].iloc[i]:
            candidate_splits.append((dataset[feature].iloc[i-1] + dataset[feature].iloc[i])/2)
            current_target = dataset[target].iloc[i]

    splits = generate_every_pair_from_list(candidate_splits, max_range_splits)
    
    # Finding the split that minimizes the information gain component 
    for split in splits:
        split_dataset = discretize_dataset(dataset.copy(), feature, split)
        aux_entropy = 0
        for value, count in split_dataset[feature].value_counts().items():
            aux_entropy += count*entropy(split_dataset.loc[split_dataset[feature] == value], target)
        aux_entropy = aux_entropy / split_dataset.shape[0]
            
        if (aux_entropy < min_entropy):
            min_entropy = aux_entropy
            best_values = split
            
    return (min_entropy,best_values)

Por su parte, la función `generate_every_pair_from_list` se encarga de generar todas las posibles combinaciones de puntos de corte para un atributo continuo, y hemos decidido que, en caso de obtener más de 500 combinaciones, se elija un subconjunto aleatorio de tamaño 500 de estas, para evitar un tiempo de ejecución excesivo.\
Además, para alterar el dataset en tiempo de ejecución, se utiliza la función `discretize_dataset`, que se encarga de discretizar los valores de un atributo continuo en base a el o los puntos de corte obtenidos.\
A través de `get_splits`, obtenemos ya sea el mejor valor o mejor par de valores por los cuales dividir el atributo continuo, junto con el componente de entropía que se utilizará en `best_feature` para determinar el mejor atributo a elegir.

Finalmente, con todas estas funciones implementadas, obtenemos la siguiente implementación del algoritmo ID3:

In [ ]:
def id3(dataset, target, features, max_range_splits, intact_dataset):
    if len(features) == 0 or len(dataset[target].value_counts().index) == 1:
        # value_counts[0] is either the only or the most common target value left in the current dataset.
        return dataset[target].value_counts().index[0] 
    best, dataset = best_feature(dataset, target, features, continuous_features, max_range_splits)
    decision_tree = {best: {}}
    new_features = features.copy()
    new_features.remove(best)
    if(best in continuous_features):
        auxDataset = dataset
    else:
        auxDataset = intact_dataset
    for value in auxDataset[best].value_counts().index:
        examples = dataset.loc[dataset[best] == value]
        if (len(examples) == 0):
            decision_tree[best][value] = auxDataset.value_counts().index[0]
        else:
            decision_tree[best][value] = id3(examples, target, new_features, max_range_splits, intact_dataset)
    return  decision_tree


In [ ]:
import pandas as pd
import pprint as pprint

DATASET_FILE = "./lab1_dataset.csv"
dataset = pd.read_csv(DATASET_FILE, sep=",")

continuous_features = ['time', 'age', 'wtkg', 'karnof', 'preanti', 'cd40', 'cd420', 'cd80', 'cd820']
target = 'cid'
features = ['time', 'trt', 'age', 'wtkg', 'hemo', 'homo', 'drugs', 'karnof',
       'oprior', 'z30', 'zprior', 'preanti', 'race', 'gender', 'str2', 'strat',
       'symptom', 'treat', 'offtrt', 'cd40', 'cd420', 'cd80', 'cd820']

decision_tree = id3(dataset, target, features, 2)



### 2.3 Evaluación
Discutir:
    - Calculo de porcentaje
    - Division en subconjunto de entrenamiento y evaluacion- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.


Dado que por defecto algoritmo ID3 solamente aplica a ejemplares de valores discretos, en el caso de querer entrenar un modelo mediante un dataset que contiene valores continuos, será necesario discretizarlos en tiempo de ejecución. Esto es logrado mediante la división de los valores continuos utilizando *splits*, con posibles valores calculados con el promedio de dos puntos con valores objetivo distintos. Luego, dependiendo del valor del hiperparámetro *max_range_splits* (2 o 3 ), . Sin embargo, en datasets de gran tamaño, es probable que exista una gran cantidad de posibles splits, por lo cual es necesario identificar 

Cosas a mencionar:
    -Max split y discretizacion

## 2.3 Evaluación
Discutir:
    - Calculo de porcentaje
    - Division en subconjunto de entrenamiento y evaluacion- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.